In [1]:
%%time
import argparse
import os.path as osp

import torch
from torch_geometric.datasets import PPI
from torch_geometric.data import DataLoader
from torch_geometric.nn import GATConv
from sklearn.metrics import f1_score

# parser = argparse.ArgumentParser()
# parser.add_argument('--model', type=str, default='GeniePathLazy')
# args = parser.parse_args()
# assert args.model in ['GeniePath', 'GeniePathLazy']

path = osp.join('./', 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

dim = 256
lstm_hidden = 256
layer_num = 4


class Breadth(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Breadth, self).__init__()
        self.gatconv = GATConv(in_dim, out_dim, heads=1)#这里in_dim和out_dim都=dim=256
        # self.gatconv = GATConv(256, 256, heads=1)

    def forward(self, x, edge_index):
        x = torch.tanh(self.gatconv(x, edge_index))
        return x


class Depth(torch.nn.Module):
    def __init__(self, in_dim, hidden):
        super(Depth, self).__init__()
        self.lstm = torch.nn.LSTM(in_dim, hidden, 1, bias=False)

    def forward(self, x, h, c):
        x, (h, c) = self.lstm(x, (h, c))
        return x, (h, c)


class GeniePathLayer(torch.nn.Module):
    def __init__(self, in_dim):
        super(GeniePathLayer, self).__init__()
        self.breadth_func = Breadth(in_dim, dim)
        self.depth_func = Depth(dim, lstm_hidden)

    def forward(self, x, edge_index, h, c):
        x = self.breadth_func(x, edge_index)
        x = x[None, :]
        x, (h, c) = self.depth_func(x, h, c)
        x = x[0]
        return x, (h, c)


class GeniePath(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePath, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.gplayers = torch.nn.ModuleList(
            [GeniePathLayer(dim) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        for i, l in enumerate(self.gplayers):
            x, (h, c) = self.gplayers[i](x, edge_index, h, c)
        x = self.lin2(x)
        return x


class GeniePathLazy(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePathLazy, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.breadths = torch.nn.ModuleList(
            [Breadth(dim, dim) for i in range(layer_num)])
        self.depths = torch.nn.ModuleList(
            [Depth(dim * 2, lstm_hidden) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        h_tmps = []
        for i, l in enumerate(self.breadths):
            h_tmps.append(self.breadths[i](x, edge_index))
        x = x[None, :]
        for i, l in enumerate(self.depths):
            in_cat = torch.cat((h_tmps[i][None, :], x), -1)
            x, (h, c) = self.depths[i](in_cat, h, c)
        x = self.lin2(x[0])
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'GeniePath': GeniePath, 'GeniePathLazy': GeniePathLazy}
model = GeniePathLazy(train_dataset.num_features,
                           train_dataset.num_classes).to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        num_graphs = data.num_graphs
        data.batch = None
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        with torch.no_grad():
            out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0

losslist_ppigeniepath=[]
for epoch in range(1, 101):
    loss = train()
    losslist_ppigeniepath.append(loss)
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'.format(
        epoch, loss, val_f1, test_f1))
sum([torch.numel(param) for param in model.parameters()])

Epoch: 01, Loss: 0.5630, Val: 0.4852, Test: 0.4873
Epoch: 02, Loss: 0.5201, Val: 0.5218, Test: 0.5275
Epoch: 03, Loss: 0.4997, Val: 0.5411, Test: 0.5490
Epoch: 04, Loss: 0.4687, Val: 0.5469, Test: 0.5601
Epoch: 05, Loss: 0.4363, Val: 0.6446, Test: 0.6601
Epoch: 06, Loss: 0.3932, Val: 0.6743, Test: 0.6909
Epoch: 07, Loss: 0.3630, Val: 0.6897, Test: 0.7072
Epoch: 08, Loss: 0.3205, Val: 0.7445, Test: 0.7641
Epoch: 09, Loss: 0.2753, Val: 0.7942, Test: 0.8128
Epoch: 10, Loss: 0.2427, Val: 0.7981, Test: 0.8196
Epoch: 11, Loss: 0.2212, Val: 0.8278, Test: 0.8462
Epoch: 12, Loss: 0.2024, Val: 0.8436, Test: 0.8614
Epoch: 13, Loss: 0.1851, Val: 0.8635, Test: 0.8835
Epoch: 14, Loss: 0.1683, Val: 0.8674, Test: 0.8880
Epoch: 15, Loss: 0.1474, Val: 0.8811, Test: 0.9006
Epoch: 16, Loss: 0.1284, Val: 0.9051, Test: 0.9222
Epoch: 17, Loss: 0.1126, Val: 0.9120, Test: 0.9296
Epoch: 18, Loss: 0.1106, Val: 0.9131, Test: 0.9280
Epoch: 19, Loss: 0.1034, Val: 0.9153, Test: 0.9304
Epoch: 20, Loss: 0.0963, Val: 0

3455097

In [ ]:
%%time
%matplotlib inline
import argparse
import os.path as osp

import torch
from torch_geometric.datasets import PPI
from torch_geometric.data import DataLoader
from torch_geometric.nn import GATConv
from torch_geometric.nn import AGNNConv

from sklearn.metrics import f1_score

# parser = argparse.ArgumentParser()
# parser.add_argument('--model', type=str, default='GeniePathLazy')
# args = parser.parse_args()
# assert args.model in ['GeniePath', 'GeniePathLazy']

path = osp.join('./', 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

dim = 256
lstm_hidden = 256
layer_num = 4

# class agnnn(torch.nn.Module):
#     def __init__(self,in_dim,out_dim):
#         super(agnnn, self).__init__()
#         self.lin1 = torch.nn.Linear(in_dim, 16)
#         self.prop1 = AGNNConv(requires_grad=False)
#         self.prop2 = AGNNConv(requires_grad=True)
#         self.lin2 = torch.nn.Linear(16, out_dim)

#     def forward(self):
#         x = F.dropout(data.x, training=self.training)
#         x = F.relu(self.lin1(x))
#         x = self.prop1(x, edge_index)
#         x = self.prop2(x, edge_index)
#         x = F.dropout(x, training=self.training)
#         x = self.lin2(x)
#         return x
        
class Breadth(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Breadth, self).__init__()
        self.gatconv = AGNNConv(requires_grad=True)

    def forward(self, x, edge_index):
        x = torch.tanh(self.gatconv(x, edge_index))
        return x



class Depth(torch.nn.Module):
    def __init__(self, in_dim, hidden):
        super(Depth, self).__init__()
        self.lstm = torch.nn.LSTM(in_dim, hidden, 1, bias=False)

    def forward(self, x, h, c):
        x, (h, c) = self.lstm(x, (h, c))
        return x, (h, c)


class GeniePathLayer(torch.nn.Module):
    def __init__(self, in_dim):
        super(GeniePathLayer, self).__init__()
        self.breadth_func = Breadth(in_dim, dim)
        self.depth_func = Depth(dim, lstm_hidden)

    def forward(self, x, edge_index, h, c):
        x = self.breadth_func(x, edge_index)
        x = x[None, :]
        x, (h, c) = self.depth_func(x, h, c)
        x = x[0]
        return x, (h, c)


class GeniePath(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePath, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.gplayers = torch.nn.ModuleList(
            [GeniePathLayer(dim) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        for i, l in enumerate(self.gplayers):
            x, (h, c) = self.gplayers[i](x, edge_index, h, c)
        x = self.lin2(x)
        return x


class GeniePathLazy(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePathLazy, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.breadths = torch.nn.ModuleList(
            [Breadth(dim, dim) for i in range(layer_num)])
        self.depths = torch.nn.ModuleList(
            [Depth(dim * 2, lstm_hidden) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        h_tmps = []
        for i, l in enumerate(self.breadths):
            h_tmps.append(self.breadths[i](x, edge_index))
        x = x[None, :]
        for i, l in enumerate(self.depths):
            in_cat = torch.cat((h_tmps[i][None, :], x), -1)
            x, (h, c) = self.depths[i](in_cat, h, c)
        x = self.lin2(x[0])
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'GeniePath': GeniePath, 'GeniePathLazy': GeniePathLazy}
model = GeniePathLazy(train_dataset.num_features,
                           train_dataset.num_classes).to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        num_graphs = data.num_graphs
        data.batch = None
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        with torch.no_grad():
            out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0

losslist_sgenieagnn=[]

for epoch in range(1, 101):
    loss = train()
    losslist_sgenieagnn.append(loss)
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'.format(
        epoch, loss, val_f1, test_f1))
sum([torch.numel(param) for param in model.parameters()])

Epoch: 01, Loss: 0.5722, Val: 0.4580, Test: 0.4604
Epoch: 02, Loss: 0.5275, Val: 0.4911, Test: 0.4983
Epoch: 03, Loss: 0.4927, Val: 0.5555, Test: 0.5663
Epoch: 04, Loss: 0.4489, Val: 0.6030, Test: 0.6157
Epoch: 05, Loss: 0.3979, Val: 0.6771, Test: 0.6943
Epoch: 06, Loss: 0.3384, Val: 0.7380, Test: 0.7580
Epoch: 07, Loss: 0.2798, Val: 0.7862, Test: 0.8069
Epoch: 08, Loss: 0.2263, Val: 0.8340, Test: 0.8554
Epoch: 09, Loss: 0.1794, Val: 0.8663, Test: 0.8877
Epoch: 10, Loss: 0.1437, Val: 0.8936, Test: 0.9135
Epoch: 11, Loss: 0.1127, Val: 0.9150, Test: 0.9335
Epoch: 12, Loss: 0.0910, Val: 0.9256, Test: 0.9432
Epoch: 13, Loss: 0.0749, Val: 0.9358, Test: 0.9511
Epoch: 14, Loss: 0.0655, Val: 0.9428, Test: 0.9585
Epoch: 15, Loss: 0.0561, Val: 0.9487, Test: 0.9632
Epoch: 16, Loss: 0.0490, Val: 0.9533, Test: 0.9659
Epoch: 17, Loss: 0.0441, Val: 0.9549, Test: 0.9688
Epoch: 18, Loss: 0.0398, Val: 0.9581, Test: 0.9713
Epoch: 19, Loss: 0.0367, Val: 0.9598, Test: 0.9718
Epoch: 20, Loss: 0.0336, Val: 0

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline

plt.plot(losslist_ppigeniepath,label="geniepath")

plt.plot(losslist_sgenieagnn,label="mymodel")


plt.legend(loc=0, ncol=1) 
plt.savefig('./ppi_genie_sgenie对比.jpg')
plt.show()